In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
tqdm.pandas()
from pathlib import Path

from one.api import ONE

import sys
sys.path.append('/home/crombie/code/ibl_photometry/src')
from iblphotometry.loaders import KceniaLoader

from iblnm.io import LOCAL_CACHE

In [2]:
one = ONE(cache_dir=LOCAL_CACHE)

/home/crombie/mnt/ccu-iblserver/datasets.pqt does not appear to be a valid table. Skipping


In [3]:
df_sessions = pd.read_parquet('metadata/sessions.pqt')#.query('local_photometry == True')

In [4]:
df_sessions = df_sessions[df_sessions['target'].apply(len) == 1]  # Filter for sessions with only one fiber
df_sessions['target_NM'] = df_sessions.apply(lambda x: f"{x['target'][0]}-{x['NM']}", axis='columns')
df_sessions['target_NM'].value_counts()

target_NM
DR-5HT     617
LC-NE      432
VTA-DA     308
SNc-DA     279
NBM-ACh    132
MR-5HT      53
Name: count, dtype: int64

In [42]:
# Pick a target
target_NM = ['VTA-DA', 'SNc-DA']
df_target = df_sessions.query('target_NM.isin(@target_NM) & (local_photometry == True) & (session_type != "habituation")')
df_target.groupby(['subject', 'session_type']).count()['session_n']

subject    session_type
ZFM-03447  biased           9
           training        21
ZFM-03448  training        36
ZFM-04019  biased          41
           training        11
ZFM-04026  biased          12
           training        22
Name: session_n, dtype: int64

In [43]:
def _check_trials_table_fpaths(session, one=None):
    assert one is not None
    session_fpath = one.eid2path(session['eid']) 
    if session_fpath is None:
        print("WARNING: session fpath does not exist {session['eid']}")
        return False
    trials_fpaths = [
        session_fpath / 'alf' / 'task_00' / '_ibl_trials.table.pqt',
        session_fpath / 'alf' / '_ibl_trials.table.pqt'
    ]
    for fpath in trials_fpaths:
        if fpath.exists():
            return fpath
    return False

df_target['local_trials'] = df_target.progress_apply(_check_trials_table_fpaths, one=one, axis='columns')
df_target = df_target.query('local_trials != False')
df_target.groupby(['subject', 'session_type']).count()['session_n']

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152/152 [00:02<00:00, 55.28it/s]
/tmp/ipykernel_48603/1422037292.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['local_trials'] = df_target.progress_apply(_check_trials_table_fpaths, one=one, axis='columns')


subject    session_type
ZFM-03447  biased           9
           training        21
ZFM-03448  training        36
ZFM-04019  biased          41
           training        11
ZFM-04026  biased          12
           training        22
Name: session_n, dtype: int64

In [8]:
subject = 'ZFM-04019'
df_target = df_target.query('subject == @subject')

In [44]:
# cache = Path(f'{target_NM}/')
cache = Path(f'DA/')

In [45]:
columns_to_keep = ['eid', 'subject', 'start_time', 'task_protocol', 'session_type', 'session_n', 'NM', 'target', 'roi'] 
df_target[columns_to_keep].to_parquet(cache / "sessions.pqt")

In [47]:
for idx, session in tqdm(df_target.iterrows(), total=len(df_target)):
    try:
        loader = KceniaLoader(one)
        photometry = loader.load_photometry_data(session['eid'])
        gcamp = photometry['raw_calcium']
        iso = photometry['raw_isosbestic']
    except Exception as e:
        print(Exception)
        print(session['eid'])
        continue
    df_trials = pd.read_parquet(session['local_trials'])
    df_trials.to_parquet(cache / f"{session['eid']}_trials.pqt")
    gcamp.to_parquet(cache / f"{session['eid']}_gcamp.pqt")
    iso.to_parquet(cache / f"{session['eid']}_isosbestic.pqt")

 12%|███████████████                                                                                                                | 18/152 [00:20<03:39,  1.64s/it]

<class 'Exception'>
2712e1d5-bee5-4e6c-ac2d-8e5108e8b510


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 152/152 [06:39<00:00,  2.63s/it]


In [12]:
df_target

,eid,subject,start_time,number,lab,projects,url,task_protocol,session_type,strain,...,alf/photometry/photometryROI.locations.pqt,raw_photometry_data/_neurophotometrics_fpData.channels.csv,raw_photometry_data/_neurophotometrics_fpData.raw.pqt,remote_photometry,local_photometry,roi,target,session_n,target_NM,local_trials
479,59dada8a-a1c2-4ab5-bc10-b1baefcaaeaa,ZFM-04019,2022-03-18T11:01:22.187733,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],13,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
484,ba1b77b8-a44c-4ed6-b2da-a146d85e1325,ZFM-04019,2022-03-21T11:09:23.773590,2,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],14,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
490,43a46dbc-884e-44b7-b6be-89c9bca3f45b,ZFM-04019,2022-03-22T10:53:15.216348,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],15,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
498,a5f8f077-8357-4ed4-934b-70fece1898f8,ZFM-04019,2022-03-23T13:55:36.152871,3,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[],[SNc],17,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
503,4f3e65d5-a74b-4470-84ad-e1ef40f7b9ac,ZFM-04019,2022-03-24T09:17:09.190988,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],18,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
512,7d8ed0b8-63ea-4f1f-961c-5be0b1db46dd,ZFM-04019,2022-03-25T09:09:09.302754,2,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],20,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
519,4475c850-42b2-4a13-9e06-3ec5eda13e70,ZFM-04019,2022-03-28T08:59:48.333055,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],21,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
542,d8f4b6a3-6ce3-49bd-8075-4a3d0ce737fc,ZFM-04019,2022-04-01T07:33:14.756696,2,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],25,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
559,3e7f5bf2-8e34-4cae-908d-7edb0ad0aa8a,ZFM-04019,2022-04-05T11:14:55.671163,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],27,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...
564,cfa56986-11cf-4cff-8f2e-6b8eeda6cb44,ZFM-04019,2022-04-06T08:35:27.926543,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_NPH_tasks_trainingChoiceWorld26.5.3,training,Ai148xDATCre,...,False,False,False,False,True,[Region5G],[SNc],28,SNc-DA,/home/crombie/mnt/ccu-iblserver/mainenlab/Subj...


In [34]:
eid = 'b711e28f-607a-4662-bd2e-975f1a645820'
session = df_target[df_target['eid'] == eid]
session = df_target.iloc[-2]

In [35]:
df_trials = pd.read_parquet(cache / f"{session['eid']}_trials.pqt")
gcamp = pd.read_parquet(cache / f"{session['eid']}_gcamp.pqt")
iso = pd.read_parquet(cache / f"{session['eid']}_isosbestic.pqt")

In [36]:
buffer = 15
timings = [col for col in df_trials.columns if col.endswith('_times')]
t0 = df_trials[timings].min().min()
t1 = df_trials[timings].max().max()
gcamp = gcamp.iloc[gcamp.index.searchsorted(t0 - buffer):gcamp.index.searchsorted(t1 + buffer)].copy()
iso = iso.iloc[iso.index.searchsorted(t0 - buffer):iso.index.searchsorted(t1 + buffer)].copy()

In [37]:
session['roi']

array(['Region3G'], dtype=object)

In [38]:
roi = session['roi'][0]

In [39]:
from iblnm.resp import get_responses

def _load_event_times(session, df_trials):
    session.loc['cue_times'] = df_trials['goCue_times'].values
    session.loc['movement_times'] = df_trials['firstMovement_times'].values
    session.loc['reward_times'] = df_trials.query('feedbackType == 1')['feedback_times'].values
    session.loc['omission_times'] = df_trials.query('feedbackType == -1')['feedback_times'].values
    return session

session = _load_event_times(session, df_trials)

events = ['cue', 'movement', 'reward', 'omission']
psths = []
for event in events:
    responses, tpts = get_responses(gcamp[roi], session[f'{event}_times'])
    psths.append(responses)

/tmp/ipykernel_48603/29062795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session.loc['cue_times'] = df_trials['goCue_times'].values
/tmp/ipykernel_48603/29062795.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session.loc['movement_times'] = df_trials['firstMovement_times'].values
/tmp/ipykernel_48603/29062795.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session.loc['reward_times'] = df_trials.query('feedbackType ==

In [40]:
%matplotlib tk

from iblnm.util import EVENT2COLOR
from matplotlib import gridspec
from scipy import stats

fig = plt.figure(figsize=(12, 8))
grid = gridspec.GridSpec(2, 3)

ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
ax1 = fig.add_subplot(grid[0, :])

ax1.plot(gcamp[roi])
ax1.plot(iso[roi], color='gray')
ax1.set_ylabel('Signal (a.u.)')
ax1.set_title(f"{session['subject']}, {session['session_type']}, session {session['session_n']}")

colors = [EVENT2COLOR[event] for event in events]
for event, responses, color, ax in zip(events, psths, colors, [ax2, ax3, ax4, ax4]):
    ax.plot(tpts, responses.mean(axis=0), color=color, label=event)
    ax.plot(tpts, responses.mean(axis=0) - stats.sem(responses, axis=0), ls='--', color=color)
    ax.plot(tpts, responses.mean(axis=0) + stats.sem(responses, axis=0), ls='--', color=color)
    ax.axvline(0, ls='--', color='black', alpha=0.5)
    ax.axhline(0, ls='--', color='gray', alpha=0.5)
    ax.set_xlabel('Time (s)')
    if event != 'omission':
        ax.set_title(event.capitalize())
    ax.ticklabel_format(axis='y', style='sci', scilimits=[-2, 2])

In [41]:
session

eid                                                                        f8980936-90f3-4285-8ec2-a9a051f7f7e4
subject                                                                                               ZFM-04019
start_time                                                                           2023-04-04T08:52:56.274002
number                                                                                                        2
lab                                                                                                   mainenlab
projects                                                                                  [ibl_fibrephotometry]
url                                                           https://alyx.internationalbrainlab.org/session...
task_protocol                                                               _iblrig_NPH_tasks_biasedChoiceWorld
session_type                                                                                            